# GBRT Model: Gradient Boosting Regression Tree

Gradient boosting fits residuals by sequentially adding weak learners (decision trees):

- **Boosting**: Each tree fits the residuals of the previous tree
- **Gradient Descent**: Uses gradient descent to optimize the loss function
- **Strong Non-linear Modeling**: Usually performs better than Random Forest

Main Hyperparameters:
- `n_estimators`: Number of trees
- `learning_rate`: Learning rate (shrinkage)
- `max_depth`: Maximum depth of trees
- `subsample`: Proportion of samples used by each tree


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
import sys
import os

# Import utility functions
sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import (
    load_data, 
    prepare_features_target, 
    calculate_r2_os, 
    build_portfolio_returns,
    calculate_prediction_metrics
)
from TimeBasedCV import TimeBasedCV

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


## 1. Load Data


In [2]:
# Load data
df = load_data('ger_factor_data_from2003.csv')

print(f"Data shape: {df.shape}")
print(f"Date range: {df['eom'].min()} to {df['eom'].max()}")
print(f"Number of stocks: {df['id'].nunique()}")

# Prepare features and target
X, y, metadata, feature_names = prepare_features_target(df)
print(f"\nNumber of features: {len(feature_names)}")
print(f"Number of samples: {len(X)}")


Data shape: (604564, 23)
Date range: 2019-01-01 00:00:00 to 2024-11-01 00:00:00
Number of stocks: 13176

Number of features: 16
Number of samples: 604564


## 2. Setup Time Series Cross-Validation


In [3]:
# Create time series cross-validation object
# Adjust based on data length: data is about 6 years, use smaller train/val/test periods
cv = TimeBasedCV(
    train_period=24,   # 2 years training period
    val_period=6,      # 6 months validation period
    test_period=6,     # 6 months test period
    freq='months'
)

# Prepare dataframe for CV
cv_df = metadata.copy()
cv_df['eom'] = pd.to_datetime(cv_df['eom'])

# Set first split date
min_date = cv_df['eom'].min()
max_date = cv_df['eom'].max()
total_months = (max_date.year - min_date.year) * 12 + (max_date.month - min_date.month) + 1
print(f"Total time span: {total_months} months ({min_date.date()} to {max_date.date()})")

first_split_date = (min_date + pd.DateOffset(months=30)).date()
second_split_date = (min_date + pd.DateOffset(months=36)).date()

# Generate cross-validation folds
folds = cv.split(cv_df, first_split_date, second_split_date, date_column='eom', gap=0)

print(f"Generated {len(folds)} cross-validation folds")


Total time span: 71 months (2019-01-01 to 2024-11-01)


Train period: 2019-07-01 - 2021-07-01 ,val period: 2021-07-01 - 2022-01-01 , Test period 2022-01-01 - 2022-07-01 # train records 184419 ,# val records 53091 , # test records 55458
Train period: 2019-10-01 - 2021-10-01 ,val period: 2021-10-01 - 2022-04-01 , Test period 2022-04-01 - 2022-10-01 # train records 188377 ,# val records 54619 , # test records 55511
Train period: 2020-01-01 - 2022-01-01 ,val period: 2022-01-01 - 2022-07-01 , Test period 2022-07-01 - 2023-01-01 # train records 193202 ,# val records 55458 , # test records 55493
Train period: 2020-04-01 - 2022-04-01 ,val period: 2022-04-01 - 2022-10-01 , Test period 2022-10-01 - 2023-04-01 # train records 198473 ,# val records 55511 , # test records 55300
Train period: 2020-07-01 - 2022-07-01 ,val period: 2022-07-01 - 2023-01-01 , Test period 2023-01-01 - 2023-07-01 # train records 203973 ,# val records 55493 , # test records 54766
Train period: 2020-10-01 - 2022-10-01 ,val period: 2022-10-01 - 2023-04-01 , Test period 2023-04-01 

## 3. Define Model Configuration (Fixed Parameters)

**Strategy**: Use empirically stable configuration (Method A), skip hyperparameter search

For asset pricing scenarios (large samples + high-dimensional features), use the following fixed configuration:
- `n_estimators=300`: Number of trees
- `learning_rate=0.05`: Learning rate (step size)
- `max_depth=3`: Tree depth (weak learner, consistent with boosting theory)
- `subsample=0.7`: Each tree uses 70% of samples (increases randomness, prevents overfitting)
- `min_samples_leaf=50`: At least 50 samples in leaf nodes (prevents very narrow leaves in panel data)

Characteristics of this configuration:
- Trees are not deep (3 layers), each tree is "weak", more like the weak learners required by boosting theory
- Increases randomness and prevents overfitting through subsample < 1
- min_samples_leaf=50 prevents very narrow leaves in panel data


In [4]:
# Use fixed configuration (Method A: empirically stable parameters)
FIXED_PARAMS = {
    'n_estimators': 300,
    'learning_rate': 0.05,
    'max_depth': 3,
    'subsample': 0.7,
    'min_samples_leaf': 50,
    'random_state': 42
}

print("✅ Using Method A: Fixed configuration (empirically stable parameters)")
print("\nFixed parameter configuration:")
for key, value in FIXED_PARAMS.items():
    print(f"  {key}={value}")
print("\nNote: Use these parameters directly for training, skip hyperparameter search, significantly speed up training")


✅ Using Method A: Fixed configuration (empirically stable parameters)

Fixed parameter configuration:
  n_estimators=300
  learning_rate=0.05
  max_depth=3
  subsample=0.7
  min_samples_leaf=50
  random_state=42

Note: Use these parameters directly for training, skip hyperparameter search, significantly speed up training


## 4. Train Model and Make Predictions (Using Fixed Configuration)


In [5]:
# Store all prediction results
all_predictions = []
all_actuals = []
all_dates = []
all_ids = []
feature_importances_list = []

# Train and predict for each fold
for fold_idx, (train_idx, val_idx, test_idx) in enumerate(folds):
    print(f"\nProcessing fold {fold_idx + 1}/{len(folds)}...")
    print(f"  📌 Using fixed configuration (Method A: empirically stable parameters)")
    
    # Prepare training, validation, and test data
    X_train = X.iloc[train_idx].values
    y_train = y.iloc[train_idx].values
    X_val = X.iloc[val_idx].values
    y_val = y.iloc[val_idx].values
    X_test = X.iloc[test_idx].values
    y_test = y.iloc[test_idx].values
    
    # Standardize features (GBRT usually doesn't need this, but for consistency)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Use fixed configuration to train on training+validation set
    X_trainval = np.vstack([X_train_scaled, X_val_scaled])
    y_trainval = np.hstack([y_train, y_val])
    
    model = GradientBoostingRegressor(
        n_estimators=FIXED_PARAMS['n_estimators'],
        learning_rate=FIXED_PARAMS['learning_rate'],
        max_depth=FIXED_PARAMS['max_depth'],
        subsample=FIXED_PARAMS['subsample'],
        min_samples_leaf=FIXED_PARAMS['min_samples_leaf'],
        random_state=FIXED_PARAMS['random_state']
    )
    
    # Train model
    print(f"  Training... (n_estimators={FIXED_PARAMS['n_estimators']}, learning_rate={FIXED_PARAMS['learning_rate']}, max_depth={FIXED_PARAMS['max_depth']})")
    model.fit(X_trainval, y_trainval)
    
    # Save feature importances
    feature_importances_list.append(model.feature_importances_)
    
    # Predict on test set
    y_pred = model.predict(X_test_scaled)
    
    # Save results
    all_predictions.extend(y_pred)
    all_actuals.extend(y_test)
    all_dates.extend(metadata.iloc[test_idx]['eom'].values)
    all_ids.extend(metadata.iloc[test_idx]['id'].values)
    
    # Print fold performance
    r2_fold = calculate_r2_os(y_test, y_pred)
    print(f"  📈 Test R²_OS: {r2_fold:.4f}")

print(f"\n✅ Completed predictions for all folds!")
print(f"Total prediction samples: {len(all_predictions)}")
print(f"\nFixed parameter configuration used:")
for key, value in FIXED_PARAMS.items():
    print(f"  {key}={value}")



Processing fold 1/10...
  📌 Using fixed configuration (Method A: empirically stable parameters)
  Training... (n_estimators=300, learning_rate=0.05, max_depth=3)


KeyboardInterrupt: 

In [ ]:
# Calculate overall performance metrics
metrics = calculate_prediction_metrics(all_actuals, all_predictions)

print("=" * 50)
print("GBRT Model Prediction Performance Metrics")
print("=" * 50)
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key:20s}: {value:8.4f}")
    else:
        print(f"{key:20s}: {value}")
print("=" * 50)

# Calculate average feature importance
avg_feature_importance = np.mean(feature_importances_list, axis=0)
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': avg_feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance_df.head(20))


## 5. Build Portfolio and Calculate Returns


In [ ]:
# Build portfolio returns
portfolio_df, summary_stats = build_portfolio_returns(
    all_actuals, 
    all_predictions, 
    all_dates, 
    all_ids, 
    n_deciles=10
)

print("\n" + "=" * 50)
print("Long-Short Portfolio Performance (GBRT Model)")
print("=" * 50)
for key, value in summary_stats.items():
    if isinstance(value, float):
        print(f"{key:25s}: {value:10.4f}")
    else:
        print(f"{key:25s}: {value}")
print("=" * 50)


## 6. Visualize Results


In [ ]:
# 1. Cumulative return curve
if 'long_short' in portfolio_df.columns:
    portfolio_df['cumulative_return'] = (1 + portfolio_df['long_short']).cumprod() - 1
    
    plt.figure(figsize=(14, 6))
    plt.plot(portfolio_df['date'], portfolio_df['cumulative_return'], linewidth=2, color='purple')
    plt.title('GBRT Model: Long-Short Portfolio Cumulative Returns', fontsize=14, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Cumulative Return', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

# 2. Predicted vs Actual scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(all_actuals, all_predictions, alpha=0.1, s=1)
plt.plot([np.min(all_actuals), np.max(all_actuals)], 
         [np.min(all_actuals), np.max(all_actuals)], 
         'r--', linewidth=2, label='Perfect Prediction Line')
plt.xlabel('Actual Returns', fontsize=12)
plt.ylabel('Predicted Returns', fontsize=12)
plt.title(f'GBRT Model: Predicted vs Actual\nR²_OS = {metrics["r2_os"]:.4f}', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Feature importance visualization (top 20)
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(20)
plt.barh(range(len(top_features)), top_features['importance'], color='purple')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance', fontsize=12)
plt.title('GBRT Model: Top 20 Most Important Features', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()
